## 0. Requirement

In [ ]:
## jalankan ini dahulu sebelum running
%pip install -qq -r requirements.txt

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

## 1. Pengenalan

In [ ]:
from bs4 import BeautifulSoup
with open("./files/latihan.html") as f:
    soup = BeautifulSoup(f, 'html.parser')

In [ ]:
soup

In [ ]:
print(soup.prettify())

In [ ]:
soup.find('title')

In [ ]:
soup.find('title').get_text()

In [ ]:
soup.find('p').get_text(strip=True)

In [ ]:
soup.find('a').get_text()

In [ ]:
soup.find('a', href=True)

In [ ]:
soup.find('a', href=True).get('href')

## 2. Let's dive into the real case

In [ ]:
## check if there is a response, if it's 200, we are good to go, if only it's ethical
## check robots.txt

s = requests.Session()
url = 'https://google.com/maps'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}
response = s.get(url, headers=headers, timeout=60)
print(response.status_code)

In [ ]:
## check robots.txt
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://news.detik.com/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'
}
response = s.get(url, headers=headers, timeout=60)
print(response.status_code)

### CSS Selector

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
soup.find('div', class_= 'replace_title')

In [ ]:
len(soup.find_all('div', class_= 'replace_title'))

In [ ]:
soup.find_all('div', class_= 'replace_title').get_text(strip=True) #error, karena bentuknya list

In [ ]:
soup.find_all('div', class_= 'replace_title')[2].get_text(strip=True)

In [ ]:
titles = soup.find_all('div', class_= 'replace_title')
for title in titles:
    print(title.get_text(strip=True), "\n")

In [ ]:
# mengambil 3 fitur: judul, tanggal, dan link berita
print(soup.find('div', class_= 'section nhl').prettify())

In [ ]:
section = soup.find('div', class_= 'section nhl')
print(section.find_all('div', class_= 'media__text')[0].prettify())

In [ ]:
#judul, tanggal, link
beritas = section.find_all('div', class_= 'media__text')

for berita in beritas:
    judul = berita.find('h3', class_='media__title').get_text(strip=True)
    tanggal = berita.find('div', class_='media__date').get_text(strip=True)
    link = berita.find('a', href=True).get('href')
    print(f"Judul: {judul}\nTanggal: {tanggal}\nLink: {link}\n")        

In [ ]:
beritas = section.find_all('div', class_= 'media__text')
hasil = []

for berita in beritas:
    judul = berita.find('h3', class_='media__title').get_text(strip=True)
    tanggal = berita.find('div', class_='media__date').get_text(strip=True)
    link = berita.find('a', href=True).get('href')
    hasil.append({
        "judul_berita": judul,
        "waktu_upload": tanggal,
        "link_berita": link

    })
    
df = pd.DataFrame(hasil)
df.info()

In [ ]:
df.head()

In [ ]:
df.to_csv("./files/hasil_scraping_awal.csv")

### Xpath

In [ ]:
import requests
from lxml import html

url = "https://news.detik.com/"
response = requests.get(url)
tree = html.fromstring(response.content)

In [ ]:
tree.xpath('//nav[@class="navbar-first"]/ul[@class="nav font-heading"]/li/a/text()') # menu navigasi

In [ ]:
tree.xpath("/html/body/div[5]/div[2]/nav/ul/li/a/text()")  # menu navigasi copy xpath from browser

In [ ]:
tree.xpath("/html/body/div[5]/div[2]/nav/ul/li[2]/a/text()")  # menu navigasi kedua

## 3. Exercise at Home: Coba sesuaikan script berikut agar bisa jalan seperti bagian CSS Selector

In [ ]:
import requests
from lxml import html
import pandas as pd

url = "https://news.detik.com/"
response = requests.get(url)
tree = html.fromstring(response.content)

beritas = tree.xpath('//div[contains(@class, "media__text")]')

hasil = []
for berita in beritas:
        title_raw = berita.xpath('//h3[@class="media__title"]/a/text()')
        title = [t.strip() for t in title_raw if t.strip()] if title else ""

        upload_raw = berita.xpath('.//div[@class, "media__date"]/span//text()')
        upload = upload_raw[0].strip() if upload_raw else ""

        link_raw = berita.xpath('.//h3[@class, "media__title"]/a/@href')
        link = link_raw[0].strip() if link_raw else ""
        
        # Skip jika semua kosong
        if not title and not upload and not link:
            continue

        hasil.append({
            "judul_berita": title,
            "waktu_upload": upload,
            "tautan_berita": link
        })

df = pd.DataFrame(hasil)
df.info()